# 앙상블(Ensemble Learning)

- 여러 개의 의사결정트리를 결합하여 하나의 결정트리보다 더 좋은 성능을 내는 머신러닝 학습기법
- 앙상블 학습의 핵심은 약한 분류기를 병렬(배깅) 또는 직렬(부스팅)로 결합하여 강력한 분류기로 만드는 것이다.
- 정형데이터의 분류 중 가장 뛰어난 기법
- 종류
  1. 배깅: 동일 알고리즘을 병렬로 사용
  2. 부스팅: 동일 알고리즘을 직렬로 사용
  3. 보팅: 다른 알고리즘을 병렬로 사용

## 랜덤 포레스트(Random Forest)

- 앙상블 학습의 대표 주자
- 안정적인 성능을 보여주기 때문에 가장 많이 쓰임
- 결정트리를 랜덤하게 만들어 결정트리 숲을 만든다.
- 랜덤 포레스트는 각각의 트리를 훈련하기 위한 데이터를 랜덤하게 만든다.
- 랜덤 포레스트는 랜덤하게 선택한샘플과 특성을 사용하기 떄문에 훈련 세트에 과대적합되는 것을 막아주고 검증세트와 테스트세트에서 안정적인 성능을 얻는다.
- 부트스트램 샘플: 부트스트램 방식으로 샘플링하여 분류한 데이터
  - 중복된 데이터가 있을 수 있다.
  - 훈련세트의 크기와 같게 만든다.


# Module loading

In [1]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install mglearn
import mglearn

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 8.7 MB/s eta 0:00:00


In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


- RandomForestClassifier: 분류
- RandomForestResressor: 회귀
- sklearn의 랜덤포레스트는 100개의 결정트리를 훈련하는 방식
- 분류일 때 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스로 결과 예측하고, 회귀일 때는 각 트리의 예측값을 평균하여 결과 예측
- 트리알고리즘의 가장 큰 단점은 과대적합이 발생한다는 것이다.
- 랜덤포레스트는 과대적합을 막아줌

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# return_train_score=True: 검증세트 뿐만아니라 훈련세트의 점수도 같이 반환한다.
scores = cross_validate(rf, train_input, train_target, n_jobs=-1, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 과대적합

0.9973541965122431 0.8905151032797809


In [7]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- 중복이 가능하다보니 학습에 사용되지 않은 데이터는 자체적으로 검증데이터셋으로 사용될 수 있다.
- 부트스트램 샘플을 이용하여 학습하는 랜덤포레스트는 데이터의 중복을 가져올 수 있지만, 그래서 사용되지 않은 데이터가 있을 수도 있다. 그것을 OOB(Out Of Bag)이라 한다.
- OOB를 마치 검증세트처럼 사용하여 자체 평가를 하는 기능을 제공한다.

In [9]:
# oob_score 매개변수에 True(default: False)하면 oob 점수를 반환해줌
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 하이퍼파라미터 튜닝

In [10]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth':[6, 8, 10, 12],
    'min_samples_leaf':[8, 12, 18],
    'min_samples_split':[8, 16, 20]
}

model = RandomForestClassifier(n_jobs=-1, random_state=42)
gs = GridSearchCV(model, params, n_jobs= -1, cv=2)
gs.fit(train_input, train_target)

GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100]})

In [11]:
gs.best_params_

{'max_depth': 8,
 'min_samples_leaf': 8,
 'min_samples_split': 20,
 'n_estimators': 100}

In [12]:
gs.best_score_

0.8708862679167477

## Voting Classifier

In [13]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [16]:
# 개별 모델 객체 생성
# L1(Lasso), L2(Ridge) 로지스틱은 L1, L2 규제를 한번에 사용 가능
lr = LogisticRegression(solver='liblinear')
knn = KNeighborsClassifier(n_neighbors=3)

# 소프트 보팅으로 구현
## estimators=[('표시될 이름1', 모델1), ('표시될 이름2', 모델2)]
vo = VotingClassifier(estimators=[('LR', lr), ('KNN', knn)], voting='soft')

# 학습
vo.fit(train_input, train_target)

# 예측
pred = vo.predict(test_input)

# 정확도
from sklearn.metrics import accuracy_score
print('Voting 분류기 정확도:', accuracy_score(test_target, pred))

# 모델별 학습/예측/평가
models = [lr, knn]
for model in models:
  model.fit(train_input, train_target)
  pred = model.predict(test_input)
  model_name = model.__class__.__name__
  score = accuracy_score(test_target, pred)
  print(f'{model_name} 정확도: {score:.4f}')

Voting 분류기 정확도: 0.8523076923076923
LogisticRegression 정확도: 0.7754
KNeighborsClassifier 정확도: 0.8469
